In [2]:
from keras.applications.vgg16 import VGG16
model = VGG16()
import os
import cv2

import numpy as np
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications import VGG16
import warnings
warnings.filterwarnings("ignore")   


In [7]:
train_dir = "Datasets/Train"
valid_dir = "Datasets/Test"

img_width, img_height = 224, 224  # Default input size for VGG16
vgg = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))
# Extract features
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count,6))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='categorical')
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = vgg.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 240)  # Agree with our small dataset size
validation_features, validation_labels = extract_features(valid_dir, 120)


Found 240 images belonging to 6 classes.
Found 120 images belonging to 6 classes.


In [8]:
XtrainForKNN=GlobalAveragePooling2D()(train_features).numpy()
XtestForKNN=GlobalAveragePooling2D()(validation_features).numpy()
YtestforKNN=[]
for i in validation_labels:
    count=0
    for j in i:
        if j==1:
            YtestforKNN.append(count)
            break
        count+=1
YtrainforKNN=[]
for i in train_labels:
    count=0
    for j in i:
        if j==1:
            YtrainforKNN.append(count)
            break
        count+=1
  


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [9]:
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.model_selection import GridSearchCV
#create new a knn model
knn2 = KNC()
#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1, 25)}
#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)
#fit model to data
knn_gscv.fit(XtrainForKNN, YtrainforKNN)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

In [10]:
maxAccuracy=0
kbest=1
for i in np.arange(1,100,1):
    knn4=KNC(n_neighbors=i)
    knn4.fit(XtrainForKNN, YtrainforKNN)
    if maxAccuracy<knn4.score(XtestForKNN,YtestforKNN):
        maxAccuracy=knn4.score(XtestForKNN,YtestforKNN)
        kbest=i
print(maxAccuracy,kbest)

0.8916666666666667 3


In [11]:
print("K Accuracy")
for i in np.arange(1,21,1):
    knn4=KNC(n_neighbors=i)
    knn4.fit(XtrainForKNN, YtrainforKNN)
    print(i,knn4.score(XtestForKNN,YtestforKNN))
    

K Accuracy
1 0.85
2 0.8583333333333333
3 0.8916666666666667
4 0.8583333333333333
5 0.85
6 0.875
7 0.8833333333333333
8 0.875
9 0.8666666666666667
10 0.8583333333333333
11 0.8583333333333333
12 0.8583333333333333
13 0.8666666666666667
14 0.8583333333333333
15 0.875
16 0.8916666666666667
17 0.8583333333333333
18 0.8666666666666667
19 0.85
20 0.8583333333333333
